### loadData函数及测试

In [2]:
import json
import pandas as pd
import numpy as np
import os
import shutil

def loadData(user_qoe_folder, epoch):
    # 构造文件路径
    qoe_file = os.path.join(user_qoe_folder, f"user_qoe_{epoch}.csv")

    # 加载用户QoE并转换为字典格式
    if os.path.exists(qoe_file):
        qoe_df = pd.read_csv(qoe_file)
        qoe = {
            row['User ID']: eval(row['QoE']) for _, row in qoe_df.iterrows()
        }
    else:
        raise FileNotFoundError(f"文件 {qoe_file} 不存在")

    return qoe


### 测试 loadData 函数

In [3]:
# 测试函数
user_qoe_folder = "/Users/yangpeilin/NUS CE/project/mobile-env-gan/collectData2/UserQoE"
epoch = 0

qoe = loadData(user_qoe_folder, epoch)

# 设置 Pandas 显示选项，确保显示完整内容
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)     # 显示所有行
pd.set_option('display.max_colwidth', None) # 显示完整的列内容

# 打印完整的基站布局数据
print("用户QoE情况:")
print(qoe)

# 打印完整的数据速率数据和每个列表的长度
print("\n数据速率数据:")
for user_id, rates in qoe.items():
    print(f"User ID: {user_id}, QoE Length: {len(rates)} \nQoE: {rates}")

# 如果想恢复默认显示设置
# pd.reset_option('all')

用户QoE情况:
{0: [1.0, 0.72, 0.52, 0.27, 0.13, -0.05, 0.13, 0.39, 0.52, 0.65, 0.8, 0.9, 0.92, 0.72, 0.55, 0.41, 0.29, 0.04, -0.05, -0.22], 1: [0.39, 0.46, 0.37, 0.37, 0.42, 0.51, 0.47, 0.38, 0.27, 0.34, 0.35, 0.3, 0.18, 0.07, -0.03, -0.09, -0.18, -0.2, -0.11, -0.01], 2: [0.8, 0.7, 0.72, 0.75, 0.74, 0.71, 0.55, 0.39, 0.18, 0.2, 0.19, 0.14, 0.02, -0.07, -0.15, 0.19, 0.31, 0.44, 0.61, 0.67], 3: [0.74, 0.49, 0.39, 0.16, 0.07, -0.05, 0.14, 0.09, -0.02, -0.02, -0.05, -0.09, -0.17, -0.2, -0.23, -0.22, -0.26, -0.01, 0.09, 0.35], 4: [0.07, -0.12, -0.07, -0.13, -0.03, 0.01, 0.28, 0.32, 0.34, 0.46, 0.53, 0.5, 0.35, 0.35, 0.6, 1.0, 1.0, 1.0, 0.94, 1.0], 5: [0.15, 0.1, 0.05, -0.23, -0.27, -0.36, -0.22, -0.24, -0.27, -0.18, -0.07, 0.05, 0.16, 0.33, 0.57, 0.93, 1.0, 1.0, 0.87, 0.58], 6: [1.0, 1.0, 1.0, 1.0, 0.75, 0.73, 0.73, 0.67, 0.58, 0.49, 0.49, 0.44, 0.1, 0.27, 0.5, 0.86, 1.0, 1.0, 1.0, 0.74]}

数据速率数据:
User ID: 0, QoE Length: 20 
QoE: [1.0, 0.72, 0.52, 0.27, 0.13, -0.05, 0.13, 0.39, 0.52, 0.65, 0.8, 

### 计算分数

In [4]:
def qoeValue(qoe_dict, low_qoe_threshold=0.0, weights=(1.0, 0.1, 10.0)):
    # 展平所有用户的 QoE 数据
    all_qoe = np.hstack(list(qoe_dict.values()))
    
    # 计算平均 QoE
    average_qoe = np.mean(all_qoe)
    
    # 计算 QoE 方差
    qoe_variance = np.var(all_qoe)
    
    # 计算低 QoE 比例
    low_qoe_proportion = np.sum(all_qoe < low_qoe_threshold) / len(all_qoe)
    
    # 权重
    w1, w2, w3 = weights

    # 计算综合得分
    score = w1 * average_qoe - w2 * qoe_variance - w3 * low_qoe_proportion

    # 返回计算结果
    return {
        "Average QoE": average_qoe,
        "QoE Variance": qoe_variance,
        "Low QoE Proportion": low_qoe_proportion,
        "Score": score
    }


### 测试 计算分数

In [5]:
user_qoe_folder = "/Users/yangpeilin/NUS CE/project/mobile-env-gan/collectData2/UserQoE"
epoch = 222

# 加载用户 QoE 数据
qoe_dict = loadData(user_qoe_folder, epoch)

# 计算得分
result = qoeValue(qoe_dict)

# 打印结果
print("基站布局评分结果:")
for key, value in result.items():
    print(f"{key}: {value:.3f}")


基站布局评分结果:
Average QoE: 0.679
QoE Variance: 0.069
Low QoE Proportion: 0.000
Score: 0.672


### 找到最好的基站布局

In [7]:
import os
import json
import pandas as pd
import numpy as np

# 文件路径
user_qoe_folder = "/Users/yangpeilin/NUS CE/project/mobile-env-gan/collectData2/UserQoE"
output_file = "/Users/yangpeilin/NUS CE/project/mobile-env-gan/collectData2/baseStationResults.txt"

# 确保输出文件存在并清空
if os.path.exists(output_file):
    with open(output_file, 'w') as f:
        f.truncate(0)
else:
    with open(output_file, 'w') as f:
        pass

# 初始化结果存储
results = []

# 遍历所有文件并计算得分
for filename in os.listdir(user_qoe_folder):
    if filename.endswith(".csv"):
        print(f"正在处理文件: {filename}")
        file_path = os.path.join(user_qoe_folder, filename)
        epoch = int(filename.replace('.csv', '').split('_')[2])  # 提取 epoch
        try:
            # 使用 loadData 函数加载数据
            qoe_dict = loadData(user_qoe_folder, epoch)

            # 检查数据是否为空
            if not qoe_dict:
                print(f"文件 {filename} 的 QoE 数据为空，跳过。")
                continue

            # 计算布局得分
            layout_score = qoeValue(qoe_dict)
            layout_score['Epoch'] = epoch
            results.append(layout_score)
        except Exception as e:
            print(f"处理文件 {filename} 时发生错误: {e}")

# 检查 results 是否为空
if not results:
    print("没有有效的基站布局得分数据，退出。")
    with open(output_file, 'w') as f:
        f.write("没有找到有效的基站布局数据。\n")
    exit()

# 按分数排序
results.sort(key=lambda x: x['Score'], reverse=True)

# 保存结果到文件
with open(output_file, 'w') as f:
    for result in results:
        f.write(f"Epoch: {result['Epoch']}, Score: {result['Score']:.3f}, "
                f"Average QoE: {result['Average QoE']:.3f}, Variance: {result['QoE Variance']:.3f}, "
                f"Low QoE Proportion: {result['Low QoE Proportion']:.3f}\n")

# 输出最佳基站布局
best_layout = results[0]
print(f"最佳基站布局: Epoch {best_layout['Epoch']}, Score: {best_layout['Score']:.3f}")


正在处理文件: user_qoe_423.csv
正在处理文件: user_qoe_345.csv
正在处理文件: user_qoe_351.csv
正在处理文件: user_qoe_437.csv
正在处理文件: user_qoe_379.csv
正在处理文件: user_qoe_386.csv
正在处理文件: user_qoe_392.csv
正在处理文件: user_qoe_147.csv
正在处理文件: user_qoe_621.csv
正在处理文件: user_qoe_635.csv
正在处理文件: user_qoe_153.csv
正在处理文件: user_qoe_609.csv
正在处理文件: user_qoe_184.csv
正在处理文件: user_qoe_190.csv
正在处理文件: user_qoe_806.csv
正在处理文件: user_qoe_812.csv
正在处理文件: user_qoe_755.csv
正在处理文件: user_qoe_741.csv
正在处理文件: user_qoe_999.csv
正在处理文件: user_qoe_769.csv
正在处理文件: user_qoe_796.csv
正在处理文件: user_qoe_782.csv
正在处理文件: user_qoe_972.csv
正在处理文件: user_qoe_966.csv
正在处理文件: user_qoe_557.csv
正在处理文件: user_qoe_231.csv
正在处理文件: user_qoe_225.csv
正在处理文件: user_qoe_543.csv
正在处理文件: user_qoe_219.csv
正在处理文件: user_qoe_88.csv
正在处理文件: user_qoe_63.csv
正在处理文件: user_qoe_594.csv
正在处理文件: user_qoe_580.csv
正在处理文件: user_qoe_77.csv
正在处理文件: user_qoe_76.csv
正在处理文件: user_qoe_581.csv
正在处理文件: user_qoe_595.csv
正在处理文件: user_qoe_62.csv
正在处理文件: user_qoe_218.csv
正在处理文件: user_qoe_89.csv
正在处理文件